In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# In this project, I had finished some EDA process,and Utilized four predictive models( logistic regression, Support vector machine, random forest and neural network) and evaluate.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

%matplotlib inline

In [ ]:
churn=pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
churn.columns.values

In [ ]:
churn.dtypes

In [ ]:
# change totalcharges's type(object) to float64

In [ ]:
churn['TotalCharges']=pd.to_numeric(churn['TotalCharges'],errors = 'coerce')

In [ ]:
churn.isnull().sum()

In [ ]:
churn.dropna(inplace=True)

In [ ]:
# delete the first column (customerID)
churn.head()
churn_d=churn.iloc[:,1:]

In [ ]:
# transfrom yes,no to 1,0
churn_d['Churn'].replace(to_replace='Yes',value=1,inplace=True) 
churn_d['Churn'].replace(to_replace='No',value=0,inplace=True)

In [ ]:
#get_dummy 
churn_dum= pd.get_dummies(churn_d)
churn_dum.columns.values

# Distribution of our target value : churn 

In [ ]:
sns.countplot(x="Churn",data=churn)

From the result, the dataset is imbalance, which would make our accuracy measurement untenable, when 
we evaluate the model, we prefer to use recall or f1 to judge.

# Correlation Analysis

In [ ]:
plt.figure(figsize=(15,8)) 
churn_corr = churn_dum.corr()['Churn'].sort_values(ascending=False).plot(kind='bar', 
                                                                         title ='Correlation between Churn & variables'
                                                                        )
churn_corr.set_xlabel('category',fontsize=20) 
churn_corr.set_ylabel('correlation',fontsize=20)

In [ ]:
plt.figure(figsize=(10,8), dpi= 80) 
sns.heatmap(churn.corr(), xticklabels=churn.corr().columns, 
            yticklabels=churn.corr().columns, cmap='RdYlGn', center=0, annot=True)

Top 3 positive correlated variables : contract month_to_month, online_security_No, tech_support_No, 
Top 3 negative correlated variables:  tenure, contract_2_year, internet_servive_no, streaming_service_NO 

Some interetsing things:

1.contract_month_to_month and contract_2year have totally diﬀerent inﬂuences.

2.monthly charge and total_charge have opposite eﬀects on churn

3.total_charge and tenure has strong relationship (0.83)

4.absence of online security and tech support seem to be positively correlated with churn

5.customer with no internet service are less likely to churn. 

6.difference among payment methods.

...

In [ ]:
# more exploration 

 # Churn & Contract

In [ ]:
for item in churn['Contract'].unique():
    print(item)

In [ ]:
contract_types = (churn['Contract'].value_counts(normalize=True) * 100).keys().tolist() 
contract_propotion = (churn['Contract'].value_counts(normalize=True) * 100).values.tolist()
for i in range(3):
    contract_propotion[i]=round(contract_propotion[i],2)
text= ['{} %'.format(x) for x in contract_propotion]
print(contract_types)

the result shows that majority of cutsomers have month_month contract(55.11%)
Then visualize the churn rate for different groups.

In [ ]:
month_to_month =churn.loc[churn['Contract']=='Month-to-month'] 
m2m = int(round((month_to_month['Churn'].value_counts(normalize=True) * 100)['Yes']))
one_year =churn.loc[churn['Contract']=='One year'] 
oney = int(round((one_year['Churn'].value_counts(normalize=True) * 100)['Yes']))
two_year =churn.loc[churn['Contract']=='Two year'] 
twoy = int(round((two_year['Churn'].value_counts(normalize=True) * 100)['Yes']))
churn_rate = [m2m, oney, twoy] 
retention_rate = [100 - m2m, 100 - oney, 100 - twoy]
print(contract_types,'\n',contract_propotion,'\n',churn_rate,'\n',retention_rate)

In [ ]:
#Visulize the result above 

plt.figure(figsize=(8,6)) 
churn_label=['{} %'.format(x) for x in churn_rate] 
retention_label=['{} %'.format(x) for x in retention_rate]
p1=plt.bar(contract_types,churn_rate,color='yellow', hatch="*") 
p2=plt.bar(contract_types, retention_rate,bottom=churn_rate,color='#FFE4C4')
plt.ylim(0,100) 
plt.ylabel('churn and retention rate') 
plt.xlabel('contract type')

the area of yellow with star represents churn rate. According to the bar chart, people with month to month contract are much easier to churn.
M2M > 2 year > 1 year

# Churn & Charge & Contract

install plotly from terminal

****pip install plotly

https://plot.ly/python/plotly-express/

In [ ]:
import plotly.express as px 
fig = px.histogram(churn, x="Churn", y="MonthlyCharges", color='Churn', facet_col="Contract", histfunc='avg')
fig.update_layout(title_text='Average Monthly Cost by Contract Type')
fig.show()

nomatter which type of contract customer chooses, increasing average monthly charges, much easier to churn, 

# Churn & Tenure & Contract

In [ ]:
#Tenure: Number of months the customer has stayed with the company
churn["tenure"].describe()

In [ ]:
fig, a1=plt.subplots(nrows=1,ncols=1,sharey=True,figsize=(8,4))
for type in contract_types:
    for ai in [a1]:
        for title in ['M-to-M','One-year','Two-year']:
            a=sns.distplot(churn[churn['Contract']==type]['tenure'],ax=ai)
            a.set_xlabel('Tenure_month')
            a.set_ylabel('proportion of customer') 
            a.set_title('Churn & Tenure')
fig.legend(labels=['M-to-M','one-year','two-year'])

for people who only has a monthly contract tend to has a shorter tenure and longer in a two year contract,

longer time, more loyal, 

so we might need to adjust some strategies for the short contract with conditions and terms.

# Churn & Service 

In [ ]:
# build a list of service including all...

service=['PhoneService','MultipleLines','InternetService','OnlineSecurity', 
         'OnlineBackup','DeviceProtection', 'TechSupport','StreamingTV','StreamingMovies']

In [ ]:
fig,axes=plt.subplots(nrows=3,ncols=3,figsize=(12,10)) 

for i,item in enumerate(service):
    if i <3:
        ax=churn[item].value_counts().plot(kind='bar',ax=axes[i,0]) 
    if i >= 3 and i < 6:
        ax=churn[item].value_counts().plot(kind='bar',ax=axes[i-3,1]) 
    if i>=6:
        ax=churn[item].value_counts().plot(kind='bar',ax=axes[i-6,2]) 
    ax.set_title(item)

from all the figures above (except phone service)
absence of online security and tech support ... and etc. seem to be abviously positively correlated with churn. 
that when this service == 'Yes', they are less likely to churn 

data visualization results are accord with the result (1,2,3,4) of correlation analysis, 
After EDA, I would build some classification model and evaluate.

(Logistic model, random forest, support vector machine, neural netwrok)

# logistic model 

In [ ]:
churn_dum.head() # get_dummy before

In [ ]:
# choose variables 
x=churn_dum.drop(columns=['Churn']) 
y=churn_dum['Churn'] 
#normalization 
from sklearn.preprocessing import MinMaxScaler 
features=x.columns.values 
scaler=MinMaxScaler(feature_range=(0,1)) 
scaler.fit(x) 
x=pd.DataFrame(scaler.transform(x)) 
x.columns=features

In [ ]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3, random_state=101)

In [ ]:
from sklearn.linear_model import LogisticRegression 
model=LogisticRegression() 
result=model.fit(x_train,y_train)

from sklearn import metrics
lg_pred=model.predict(x_test)
print(metrics.accuracy_score(y_test,lg_pred))

In [ ]:
from sklearn.metrics import classification_report 
lg_report= classification_report(y_test, lg_pred) 
print(lg_report)

# Random Forest 

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3, random_state=101) 
model_r=RandomForestClassifier(n_estimators=100,random_state=50, oob_score=True) 
model_r.fit(x_train,y_train) 
rf_pred=model_r.predict(x_test)
print(metrics.accuracy_score(y_test,rf_pred))

In [ ]:
rf_report=classification_report(y_test, rf_pred)
print(rf_report)

# I would explore and find the most appropiate tree numbers (n_estomators)

In [ ]:
rand_score =[] 
for i in range(1,102):
    i_loop= RandomForestClassifier(n_estimators=i, random_state=101) 
    i_loop.fit(x_train,y_train) 
    loop_pre=i_loop.predict(x_test) 
    rand_score.append(metrics.accuracy_score(y_test,loop_pre))

In [ ]:
plt.plot(range(1,102),rand_score)
plt.xlabel("Range") 
plt.ylabel("accuracy score")
plt.show()

# 79 trees around

In [ ]:
model_r=RandomForestClassifier(n_estimators=79,random_state=101, oob_score=True)
model_r.fit(x_train,y_train) 
rf_new_pred=model_r.predict(x_test)
print(metrics.accuracy_score(y_test,rf_new_pred))

In [ ]:
# only a little bit improvement 

In [ ]:
rf_new_report=classification_report(y_test, rf_new_pred)
print(rf_new_report)

In [ ]:
rand_coef=model_r.feature_importances_ 
weight=pd.Series(rand_coef,index=x.columns.values)
weight.sort_values(ascending=False)[:10]

same: TOTAL charges, tenure and monthlycharges, contract .. are highly possive related with churn result 

seems no change for recall and f1 LOL

# SVM

In [ ]:
from sklearn.svm import SVC
svc_model = SVC(random_state=101)
svc_model.fit(x_train, y_train) 
accuracy_svc = svc_model.score(x_test, y_test)
print(accuracy_svc)

In [ ]:
svm_pred=svc_model.predict(x_test)

from sklearn.metrics import classification_report, confusion_matrix
svm_report=classification_report(y_test,svm_pred) 
print(svm_report)

# Neural Network

In [ ]:
x_train.head() # has been standardalized

In [ ]:
import keras

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense 
from keras import optimizers 
model_nn = Sequential() 
sgd= optimizers.SGD(lr=0.01) # (set learning rate)
# first time, no gradient descend, so might be the reason learning rate is too fast, thus...set it as 0.01

In [ ]:
model_nn.add(Dense(45, activation= 'relu', input_dim=45)) 
model_nn.add(Dense(22, activation='relu'))
model_nn.add(Dense(11, activation='relu')) 
model_nn.add(Dense(6, activation='relu')) 
model_nn.add(Dense(1,activation='sigmoid'))

In [ ]:
model_nn.compile(loss='binary_crossentropy',optimizer='SGD',metrics=['accuracy']) 
history= model_nn.fit(x_train, y_train,
                      batch_size=30, epochs=50, validation_data=(x_test,y_test))

In [ ]:
result_1=model_nn.evaluate(x_test,y_test)
result_1

In [ ]:
history_dict=history.history
history_dict.keys()

In [ ]:
loss_value=history_dict['loss'] 
val_loss_values= history_dict['val_loss']

In [ ]:
plt.clf() 
epochs=range(1,len(loss_value)+1)
plt.plot(epochs,loss_value,'bo',label='Training loss') 
plt.plot(epochs,val_loss_values,'b',label='Validation loss') 
plt.title('Training and validation loss') 
plt.xlabel('Epochs') 
plt.ylabel('loss')
plt.legend()

In [ ]:
 # Accroding to the graph, it is reasonable to use epochs=50 where has the least loss 

In [ ]:
nn_pred = model_nn.predict(x_test)
nn_pred[1:5]

In [ ]:
def threshold(nn_pred):
    lst_threshold=[] 
    for i in nn_pred:
        if i >=0.5:
            i=1 
            lst_threshold.append(i)
        else: 
            i=0 
            lst_threshold.append(i)
    return lst_threshold


for report precision, recall and f1,
Classification metrics can't handle a mix of binary and continuous targets !!!!

build threshold rule for neural network

In [ ]:
nn_pred_new = threshold(nn_pred) 
nn_pred_new[1:5]

In [ ]:
nn_report=classification_report(y_test,nn_pred_new)
print(nn_report)

In [ ]:
print('\n--------------------Logistic Regression-----------------\n',lg_report, 
      '\n----------------------Random Forest---------------------\n',rf_report, 
      '\n----------------- Support Vector Machine----------------\n',svm_report, 
      '\n---------------------Neural Network---------------------\n',nn_report)

in the view of accuracy: **Neural network(0.8109)> SVM (0.8085) > logistic regression (0.8076)>random forest(0.7981)**
In fact, i think the four models are all good predictor (over 0.8). however,because the **data is imbalanced!**, i prefer 
to compare the  recall and f1-score rather than accuracy!!

besides, the **precision is more important than recall in this customer churn prediction project, **
for we take more care on dicovering the targeted customers precisely and make eﬀective strategies.( precision: P=TP/(TP+FP) )
and classifing loyal customers into churn group would not make any loss.

Thus, logistic regression and support vector machine, neural network are a little bit better. 